In [1]:
import pennylane as qml
import numpy as np
import sys

%load_ext autoreload
%autoreload 2

In [2]:
from pennylane_ls import *
from pennylane_ls.MultiQuditDevice import MultiQuditDevice

In [3]:
from heroku_credentials import username, password
#from local_credentials  import username, password

We build up the circuit for the QAOA in Pennylane. We do this (for now) for a fixed number of qudits and fixed dimension d. We take the following instance of our benchmark suite:

exp_dict['ExpA1']={ 'jm':[0,1],\
                    'pv':[0,1],\
                    'emax':1,\
                    'emin':0 }

In [8]:
multiDeviceQAOA = MultiQuditDevice(wires=8,shots=5, username = username, password = password)

In [12]:
jm = np.array([0,1])
pv = np.array([0,1])
emax = 1
emin = 0

num_tsteps = len(jm)
dim_qudit = int(np.max(jm)+1)
num_qudit = int(2*num_tsteps + 2)
l = (dim_qudit-1)/2


@qml.qnode(multiDeviceQAOA)
def quantum_circuit_simpleQAOA(alpha=[0,0]):
    gamma = 1
    beta = 1
    for i in range(0, num_qudit):
        load(dim_qudit-1+3, wires=i)#we load atomnumbers here, therefore the -1
    for j in range(0, num_tsteps):#first part of hamiltonian
        rLz2(gamma, wires = j)
        rLz(gamma*2*(l-pv[j]), wires = j)
    for n in range(0, num_tsteps):
        rLz2(gamma,wires = n)
        rLz2(gamma, wires = num_tsteps + n)
        MultiQuditOps.LzLz(gamma*2,wires=[n, n+num_tsteps])
    return qml.expval(MultiQuditOps.Z(0))

In [13]:
res = quantum_circuit_simpleQAOA(alpha=0)

In [14]:
print(quantum_circuit_simpleQAOA.draw())

 0: ──load(4)──rLz2(1)──rLz(1)───rLz2(1)────────────╭LzLz(2)──┤ ⟨Z⟩ 
 1: ──load(4)──rLz2(1)──rLz(-1)──rLz2(1)──╭LzLz(2)──│─────────┤     
 2: ──load(4)──rLz2(1)────────────────────│─────────╰LzLz(2)──┤     
 3: ──load(4)──rLz2(1)────────────────────╰LzLz(2)────────────┤     
 4: ──load(4)─────────────────────────────────────────────────┤     
 5: ──load(4)─────────────────────────────────────────────────┤     



We have to implement the squared sum parts of the Hamiltonian and the Mixer.